In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

/data/richardlin/.conda/envs/chuenv/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


#   效能測試(第一階段)

In [2]:
Ka = pd.read_excel("川崎症病人(訓練用 第一階段).xlsx")
fever = pd.read_excel('發燒病人(訓練用 第一階段).xlsx')

In [3]:
pd.options.display.max_columns = None

In [4]:
Ka

,WBC,RBC,Hemoglobin,Hematocrit,MCH,MCHC,RDW,Platelets,Segment,Band,Lymphocyte,Monocyte,Eosinophil,Basophil,AST,ALT,CRP,UWBC,性別,年齡(日),膿尿,輸入日期
0,11.8,4.81,12.8,37.3,26.6,34.3,12.6,310.0,56.0,1.0,35.0,7.0,0.0,0.0,244.00000,175.0,68.790,5.0,1,177,0,2010-07-02
1,22.2,3.24,10.3,31.1,31.8,33.1,12.4,521.0,67.0,3.8,20.5,9.5,2.5,0.5,32.00000,34.0,18.200,6.0,0,54,1,2019-03-24
2,17.5,2.99,8.1,23.5,27.1,34.5,14.6,775.0,59.0,0.0,32.0,9.0,0.0,0.0,531.00000,597.0,51.400,33.0,1,98,1,2019-05-07
3,43.8,4.26,8.5,26.7,20.0,31.8,18.0,581.0,74.0,4.0,13.0,11.5,0.0,0.0,24.00000,9.0,302.560,111.0,0,482,1,2017-08-14
4,10.8,4.15,10.3,31.9,24.8,32.3,13.3,159.0,56.0,1.0,35.0,7.0,0.0,0.0,72.00000,47.0,136.900,6.0,0,315,0,2017-11-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1153,18.6,3.85,10.2,29.4,26.5,34.7,13.3,156.0,54.0,0.0,37.0,7.0,2.0,0.0,77.90949,53.0,183.284,20.0,0,276,0,2013-05-19
1154,15.9,4.10,10.8,32.1,26.3,33.6,13.5,358.0,58.0,0.0,26.0,11.0,5.0,0.0,36.00000,33.0,251.400,6.0,1,278,0,2017-02-04
1155,18.7,4.63,12.3,36.1,26.6,34.1,12.5,303.0,56.0,0.0,30.0,7.0,6.0,1.0,59.00000,235.0,46.600,180.0,1,459,1,2011-06-25
1156,14.3,3.90,9.9,29.8,25.4,33.2,13.2,371.0,48.0,1.5,39.0,10.0,3.0,0.0,30.00000,17.0,198.010,20.0,1,141,0,2010-08-15


In [5]:
fever

,WBC,RBC,Hemoglobin,Hematocrit,MCH,MCHC,RDW,Platelets,Segment,Band,Lymphocyte,Monocyte,Eosinophil,Basophil,AST,ALT,CRP,UWBC,性別,年齡(日),膿尿,輸入日期
0,9.7,4.33,8.2,26.3,18.9,31.2,18.8,197.0,42.0,0.0,44.0,11.0,2.0,0.0,55.000000,27.000000,10.40,3.000000,1,262,0,2017-06-05
1,11.0,4.76,8.3,28.5,17.4,29.1,17.5,241.0,64.8,0.0,30.3,4.1,0.5,0.3,44.000000,22.000000,33.70,0.000000,1,392,0,2017-10-13
2,15.6,5.29,8.8,30.8,16.6,28.6,18.7,290.0,53.0,0.0,36.0,10.0,0.0,0.0,40.000000,14.000000,22.90,0.000000,1,635,0,2018-06-13
3,3.8,4.48,7.4,26.0,16.5,28.5,18.8,170.0,17.0,0.0,69.0,14.0,0.0,0.0,51.030973,34.610109,34.92,49.853269,1,638,0,2018-06-16
4,9.6,4.42,8.7,28.3,19.7,30.7,16.2,260.0,28.0,3.0,53.0,13.0,1.0,0.0,58.000000,30.000000,6.80,3.000000,1,212,0,2017-04-16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78165,5.5,5.01,9.4,29.7,18.8,31.6,15.9,263.0,60.0,1.0,30.0,9.0,0.0,0.0,39.000000,16.000000,30.60,2.000000,1,393,0,2010-03-16
78166,4.2,5.04,9.5,30.0,18.8,31.7,16.1,185.0,30.5,0.0,63.1,5.8,0.2,0.4,42.000000,15.000000,15.50,49.853269,1,394,0,2010-03-17
78167,9.3,5.11,9.8,30.0,19.2,32.7,14.9,383.0,54.3,0.0,34.4,9.6,1.6,0.1,38.000000,17.000000,9.20,49.853269,1,465,0,2010-05-27
78168,9.5,4.09,10.9,33.3,26.7,32.7,12.0,425.0,21.0,0.0,70.0,4.0,4.0,0.0,51.030973,34.610109,5.10,49.853269,1,179,0,2018-09-09


In [ ]:
print('川崎症病人(原): ', len(Ka))
print('川崎症病人(<=5歲): ', len(Ka[Ka['年齡(日)']<=2191]))
print('川崎症病人(<5歲): ', len(Ka[Ka['年齡(日)']<2191]))

print('發燒病人(原): ', len(fever))
print('發燒病人(<=5歲): ', len(fever[fever['年齡(日)']<=2191]))
print('發燒病人(<5歲): ', len(fever[fever['年齡(日)']<2191]))
Ka = Ka[Ka['年齡(日)']<=1826]
fever = fever[fever['年齡(日)']<=1826]

In [ ]:
Ka['輸入日期(月)'] = Ka['輸入日期'].dt.month
fever['輸入日期(月)'] = fever['輸入日期'].dt.month

ka_month = pd.get_dummies(Ka['輸入日期(月)'], prefix='Month')
fever_month = pd.get_dummies(fever['輸入日期(月)'], prefix='Month')

Ka = pd.concat([Ka, ka_month], axis=1)
fever = pd.concat([fever, fever_month], axis=1)

Ka = Ka.drop(columns=['輸入日期(月)', '輸入日期'])
fever = fever.drop(columns=['輸入日期(月)', '輸入日期'])

In [ ]:
Ka

In [ ]:
fever

In [10]:
Ka = shuffle(Ka, random_state=30)
fever = shuffle(fever, random_state=30)
Ka['label'] = 1
fever['label'] = 0
all_train_data = pd.concat([Ka, fever], ignore_index=True)

In [11]:
# all_train_data.to_excel("川崎症統計用.xlsx")

In [12]:
x = all_train_data.values[:,:-1]
y = all_train_data.values[:,-1]
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8, shuffle=True, stratify=y, random_state=30)

In [ ]:
xgb = XGBClassifier()
xgb.fit(x_train, y_train)

In [ ]:
y_pred = xgb.predict(x_test)
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
print('tn: ',tn)
print('fp: ',fp)
print('fn: ',fn)
print('tp: ',tp)
print('Accuracy: ', accuracy_score(y_test, y_pred))
print('Recall: ', recall_score(y_test, y_pred))
print('Precision: ', precision_score(y_test, y_pred))
print('F1_score: ', f1_score(y_test, y_pred))

In [15]:
y_pred_proba = xgb.predict_proba(x_test)

In [16]:
print(y_pred_proba)

[[9.9997026e-01 2.9718793e-05]
 [9.9214202e-01 7.8579523e-03]
 [9.9999535e-01 4.6293317e-06]
 ...
 [9.9979728e-01 2.0274209e-04]
 [9.9999273e-01 7.2807402e-06]
 [9.9993199e-01 6.8001878e-05]]


In [17]:
thresh_list = []
accuracy_score_list = []
precision_score_list = []
recall_score_list = []
f1_score_list = []
specificity_score_list = []
npv_score_list = []
confusion_matrix_list = []
for th in range(1,101):
    th *= 0.01

    y_pred = [0 if (x[0] >= th) else 1 for x in y_pred_proba]

    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    specificity = tn / (tn+fp)
    npv = tn / (fn+tn)
    thresh_list.append(th)
    accuracy_score_list.append(accuracy_score(y_test, y_pred) * 100)
    precision_score_list.append(precision_score(y_test, y_pred) * 100)
    recall_score_list.append(recall_score(y_test, y_pred) * 100)
    f1_score_list.append(f1_score(y_test, y_pred) * 100)
    specificity_score_list.append(specificity * 100)
    npv_score_list.append(npv * 100)
    confusion_matrix_list.append([tn, fp, fn, tp])

In [ ]:
recall_standard_list = [80, 85, 90, 95, 98, 99, 100]
for recall_standard in recall_standard_list:
    high_recall_f1score = []
    high_recall_accuracy = []
    high_recall_recall = []
    high_recall_precision = []
    high_recall_specificity = []
    high_recall_npv = []
    high_recall_confusion_matrix = []
    for index, recall in enumerate(recall_score_list):
        if recall >= recall_standard:
            high_recall_f1score.append(f1_score_list[index])
            high_recall_accuracy.append(accuracy_score_list[index])
            high_recall_precision.append(precision_score_list[index])
            high_recall_recall.append(recall_score_list[index])
            high_recall_specificity.append(specificity_score_list[index])
            high_recall_npv.append(npv_score_list[index])
            high_recall_confusion_matrix.append(confusion_matrix_list[index])
    high_recall_best_f1_score_index = np.argmax(high_recall_f1score)
    print("當Recall高於", recall_standard, "% 時")
    print("且選取最高的F1-score時")
    print("Recall為為:", high_recall_recall[high_recall_best_f1_score_index], "%")
    print("Specificity為:", high_recall_specificity[high_recall_best_f1_score_index], "%")
    print("Precision為:", high_recall_precision[high_recall_best_f1_score_index], "%")
    print("NPV為:", high_recall_npv[high_recall_best_f1_score_index], "%")
    print("F1 Score為:", high_recall_f1score[high_recall_best_f1_score_index], "%")
    print("Accuracy為:", high_recall_accuracy[high_recall_best_f1_score_index], "%")
    print("TN, FP, FN, TP", high_recall_confusion_matrix[high_recall_best_f1_score_index])
    
    print("============================================")

In [ ]:
y_pred_roc = [x[1] for x in y_pred_proba]
guess_tp = np.arange(0, 1, 0.001)
guess_fp = np.arange(0, 1, 0.001)
fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred_roc, pos_label=1)
plt.plot(fpr,tpr, color='m', label = "ROC curve")
plt.plot(guess_fp,guess_tp, color='0', linestyle="-.")
plt.ylabel("True Positive Rate")
plt.xlabel("False Positive Rate")
plt.legend()
# plt.savefig('0to5 roc', dpi=500)
plt.show()

In [ ]:
test = metrics.roc_auc_score(y_test, y_pred_proba[::,1])
test